# 문제 1) Tokenizer 생성하기

In [1]:
class Tokenizer():
     
    def __init__(self):
        self.word_dict = {'oov': 0}
        self.fit_checker = False

    def preprocessing(self, sequences):
        
        import re
        
        result = []
        for i in range(len(sequences)):
            sequences[i] = re.sub(r"[^a-zA-Z0-9ㄱ-ㅎ가-힣]", ' ', sequences[i])  # 정규표현식으로 한국어/영어를 제외한 문자를 제거
            split_str = [ i.lower() for i in sequences[i].split() ]    # 문장을 단어 단위로 나누고 소문자로 변환한 후 list화
            result.append(split_str) 

        return result
    
    def fit(self, sequences):
        self.fit_checker = False

        idx = 0    # 딕셔너리에 key값으로 설정하기 위한 변수
        splited_word = self.preprocessing(sequences)
        for go_list in splited_word:     # nested list를 푼다.
            for word in go_list:         # list 안의 단어를 하나씩 꺼낸다.
                if word in self.word_dict:   # 똑같은 단어가 list 안에 중복하게 존재할 경우 
                    continue                 # 가장 먼저 추가된 단어만을 추가하기 위한 제한
                idx += 1                     # 0이 oov이기 때문에 1부터 추가 시작
                self.word_dict[word] = idx   # word_dict에 key값에 따른 value값 추가
                
        self.fit_checker = True

    def transform(self, sequences):

        tokens = self.preprocessing(sequences)
        result = tokens.copy()
        
        if self.fit_checker:
            tmp = 0
                    
            for list_token in result:
                for idx, word in enumerate(list_token):
                    if word in self.word_dict.keys():
                        result[tmp][idx] = self.word_dict[word]
                    else:
                        result[tmp][idx] = self.word_dict['oov']
                tmp += 1
            return result
        else:
            raise Exception("Tokenizer instance is not fitted yet.")

    def fit_transform(self, sequences):
        self.fit(sequences)
        result = self.transform(sequences)
        return result

In [2]:
sequences = ['I go to school.', 'I LIKE pizza!', 'we can go home!']
a = Tokenizer()
a.fit_transform(sequences)

[[1, 2, 3, 4], [1, 5, 6], [7, 8, 2, 9]]

# 문제 2) TfidfVectorizer 생성하기

In [3]:
class TfidfVectorizer:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.fit_checker = False

    def fit(self, sequences):        
        tokenized = self.tokenizer.fit_transform(sequences)
                
        from math import log
        import numpy as np
        
        self.idf = []                # idf 값을 추가하기 위한 초기 객체
        self.words = np.array([])    # tokenzied되어 있는 단어들을 저장하는 초기 객체
        N = len(tokenized)    # 문장 단위로 끊기 위한 객체 생성 
        
                                     # sent_idx = sentense_index(문장을 나누기 위한 인덱스)
        for sent_idx in tokenized:   # tokenized를 array화하여 전체 단어 숫자를 세기 위함 
            self.words = np.append(self.words, sent_idx)
        self.words = list(set(self.words)) # 중복 제거 

                                                    # word_idx = word_index(단어 하나하나를 나누기 위한 인덱스)
        for word_idx in range(len(self.words)):     # 단어들의 숫자만큼 for loop
            t = self.words[word_idx]     # (중복제거 된) 단어를 하나씩 꺼내기
            df= 0
            for doc in tokenized:   # 문장을 하니씩 꺼내기
                df += t in doc      # 문장안에 꺼낸 단어가 있을 때
                idf = log(N/(df+1)) # 공식에 집어넣기

            self.idf.append(idf)    # idf 객체에 추가하여 저장
            
        self.fit_checker = True

    def transform(self, sequences):
        
        import numpy as np
        
        if self.fit_checker:
            tokenized = self.tokenizer.transform(sequences)
        
        self.tfidf_matrix = []       # matrix값을 추가하기 위한 초기 객체        

        N = len(tokenized)

                                               # sent_idx = sentense_index(문장을 나누기 위한 인덱스)
        for sent_idx in range(N):          # 문장 단위의 for loop
            self.tfidf_matrix.append([])   # 문장의 갯수만큼의 nested list 생성
            sentens = tokenized[sent_idx]  # 문장 하나씩 slicing

            for word_idx in range(len(self.words)):
                word = self.words[word_idx]                            # self.word에 저장되어 있던 word를 하나씩 꺼냄
                if word in sentens:                                    # 만약 문장에 단어가 존재한다면
                    self.tfidf_matrix[-1].append(sentens.count(word))  # 문장안에 몇개의 단어가 있는지를 파악하여 그 수를 추가한다.
                else:
                    self.tfidf_matrix[-1].append(0)                    # 단어가 존재하지 않으면 0을 추가
        result = np.array(self.tfidf_matrix) * np.array(self.idf)      # numpy의 broadcast를 사용하여 곱셈

        return result.tolist() # nested list화
        
    def fit_transform(self, sequences):
        self.fit(sequences)
        return self.transform(sequences)

In [4]:
sequences = ['I go to school.', 'I LIKE pizza!', 'we can go home!']

se = TfidfVectorizer(Tokenizer())
se.fit(sequences)
se.transform(sequences)
se.fit_transform(sequences)

[[0.0, 0.0, 0.4054651081081644, 0.4054651081081644, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.4054651081081644, 0.4054651081081644, 0.0, 0.0, 0.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.4054651081081644,
  0.4054651081081644,
  0.4054651081081644]]